In [ ]:
def get_cannibalisation_data(listings_df_list):
    mp_dataframe = pd.DataFrame()
    for city_id in CB_PARAM_DATA.city_id_name_dict.keys():
        add_log("CANNIBALISATION STARTED FOR CITY ID : " + str(city_id))
        city_listings_data, td_unique_models = get_city_wise_data(listings_df_list, city_id)
        city_listings_data = city_listings_data.loc[~city_listings_data['model_id'].isin(td_unique_models)]

        city_depreciation_factors = CB_PARAM_DATA.city_wise_params['depreciation_data'][str(city_id)]

        for model_id in td_unique_models:
            td_active, mp_expired, mp_active = get_column_wise_data(city_listings_data, 'model_id', model_id)
            all_mp_data = pd.concat([mp_expired, mp_active], axis=0).reset_index(drop=True)
            if len(all_mp_data):
                model_depreciation_factors = city_depreciation_factors.get(str(model_id),
                                                                           city_depreciation_factors['overall'])
                set_normalised_data(all_mp_data, td_active)
                model_mp_listings = get_mp_status_data(td_active, all_mp_data, model_depreciation_factors, city_id)
                mp_dataframe = pd.concat([mp_dataframe, model_mp_listings], axis=0).reset_index(drop=True)

        add_log("CANNIBALISATION COMPLETED FOR CITY ID : " + str(city_id))
    mp_dataframe = get_valid_mp_data(mp_dataframe)
    return mp_dataframe